Keras



In [ ]:
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from scikeras.wrappers import KerasClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input


In [ ]:
# Load data
df = pd.read_csv("penguins.csv")
df.head(), df.columns, df.shape

(   culmen_length_mm  culmen_depth_mm  flipper_length_mm  body_mass_g     sex
 0              39.1             18.7              181.0       3750.0    MALE
 1              39.5             17.4              186.0       3800.0  FEMALE
 2              40.3             18.0              195.0       3250.0  FEMALE
 3               NaN              NaN                NaN          NaN     NaN
 4              36.7             19.3              193.0       3450.0  FEMALE,
 Index(['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm',
        'body_mass_g', 'sex'],
       dtype='object'),
 (344, 5))

In [ ]:
df.isnull().sum()

culmen_length_mm     2
culmen_depth_mm      2
flipper_length_mm    2
body_mass_g          2
sex                  9
dtype: int64

In [ ]:
from sklearn.impute import SimpleImputer

# Setting strategy to 'most frequent' to impute by the mean
imputer = SimpleImputer(strategy='most_frequent') # strategy can also be mean or median
df.iloc[:,:] = imputer.fit_transform(df)

df.isnull().sum()

culmen_length_mm     0
culmen_depth_mm      0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

In [ ]:
# Select features and target
X = df.drop('sex', axis=1)
y = df[['sex']]

# Convert labels to categorical one-hot encoding
y_onehot = pd.get_dummies(y, drop_first=False)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Define a function to create the Keras model
def create_model(learning_rate=0.01, neurons=10):
    model = Sequential([
        Input(shape=(4,)),  # Define the input shape explicitly
        Dense(neurons, activation='relu'),
        Dense(3, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Wrap the model so it can be used by scikit-learn
model = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=0)

# Define the grid search parameters
param_grid = {
    'model__learning_rate': [0.001, 0.01, 0.1],
    'model__neurons': [10, 20, 50],
    'batch_size': [10, 20, 30],
    'epochs': [10, 50, 100],
}

In [ ]:

# Create GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Best parameters and best score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Evaluate using the test data
score = grid_result.best_estimator_.score(X_test_scaled, y_test)
print('Test accuracy:', score)

Best: 0.887243 using {'batch_size': 20, 'epochs': 50, 'model__learning_rate': 0.1, 'model__neurons': 50}
Test accuracy: 0.8695652173913043
